In [1]:
# GEREKLI KUTUPHANELERIN EKLENMESI

import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
import torch

In [2]:
# CSV DOSYASININ OKUNMASI

dframe = pd.read_csv("/content/vodafone_data.csv", encoding='utf_8', on_bad_lines = 'skip', sep = ';')
dframe

,Explanation,Target
0,Vodafone'a ıyy diyen hayatında hiç vodafone ku...,1
1,Her yerde çekiyor diye geçtiğim güne lanet ols...,0
2,Vodafone benim için bitti artık her yerde çeki...,0
3,Vodafone Sizin de içine mi sızdılar yoksa bili...,0
4,Saf olmamak gerekir. Güvenlik açığı demek olay...,0
...,...,...
4827,Tam iki ay oldu hatalı faturam düzeltilmedi. İ...,0
4828,Müşteri hizmeti olarak arıyorsunuz 50 gb hafız...,0
4829,Izmirin ciglisinde çarşının ortasında şebekeni...,0
4830,TV’lerle uğraşmaktan bıktığım için böyle zaten...,0


In [3]:
# TITLE VE LINK SUTUNLARININ TABLODAN CIKARILMASI

#dframe = dframe.iloc[300:]
#dframe = dframe.drop(columns="Title",axis = 1)
#dframe = dframe.drop(columns="Link",axis=1)
#dframe = dframe.sample(frac = 1)
dframe

,Explanation,Target
0,Vodafone'a ıyy diyen hayatında hiç vodafone ku...,1
1,Her yerde çekiyor diye geçtiğim güne lanet ols...,0
2,Vodafone benim için bitti artık her yerde çeki...,0
3,Vodafone Sizin de içine mi sızdılar yoksa bili...,0
4,Saf olmamak gerekir. Güvenlik açığı demek olay...,0
...,...,...
4827,Tam iki ay oldu hatalı faturam düzeltilmedi. İ...,0
4828,Müşteri hizmeti olarak arıyorsunuz 50 gb hafız...,0
4829,Izmirin ciglisinde çarşının ortasında şebekeni...,0
4830,TV’lerle uğraşmaktan bıktığım için böyle zaten...,0


In [4]:
# TABLODAKI DUYGULARIN SAYISI

print("0: Olumsuz", "1: Olumlu", "2: Nötr\n")
dframe["Target"].value_counts()

0: Olumsuz 1: Olumlu 2: Nötr



,count
Target,
0,2906
1,1203
2,723


In [6]:
# METINDEN NOKTALAMA ISARETLERINI, RAKAMLARI CIKARMA

def clean_text(text):
    text = " ".join(str(text).split())
    text = text.lower()
    text = text.replace("\\n", " ")
    text = re.sub("[0-9]+", "", text)
    text = re.sub("%|(|)|-", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    return "".join(text)

dframe["Clean"] = dframe.apply(lambda row: clean_text(row["Explanation"]), axis=1)
dframe

,Explanation,Target,Clean
0,Vodafone'a ıyy diyen hayatında hiç vodafone ku...,1,vodafonea ıyy diyen hayatında hiç vodafone kul...
1,Her yerde çekiyor diye geçtiğim güne lanet ols...,0,her yerde çekiyor diye geçtiğim güne lanet ols...
2,Vodafone benim için bitti artık her yerde çeki...,0,vodafone benim için bitti artık her yerde çeki...
3,Vodafone Sizin de içine mi sızdılar yoksa bili...,0,vodafone sizin de içine mi sızdılar yoksa bili...
4,Saf olmamak gerekir. Güvenlik açığı demek olay...,0,saf olmamak gerekir güvenlik açığı demek olayı...
...,...,...,...
4827,Tam iki ay oldu hatalı faturam düzeltilmedi. İ...,0,tam iki ay oldu hatalı faturam düzeltilmedi i̇...
4828,Müşteri hizmeti olarak arıyorsunuz 50 gb hafız...,0,müşteri hizmeti olarak arıyorsunuz gb hafıza ...
4829,Izmirin ciglisinde çarşının ortasında şebekeni...,0,izmirin ciglisinde çarşının ortasında şebekeni...
4830,TV’lerle uğraşmaktan bıktığım için böyle zaten...,0,tv’lerle uğraşmaktan bıktığım için böyle zaten...


In [7]:
# SATIRLARI NUMPY DIZISI YAPMA

X = dframe["Clean"]
y = dframe["Target"]

In [8]:
# TEST VE EGITIM VERILERININ AYRILMASI

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)
X_train_data, X_val, y_train_data, y_val = train_test_split(X_train, y_train, test_size=0.5)

In [9]:
# BERT TOKENIZER EKLEME

tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
# BERT TOKENIZER UYGULAMA

max_len= 1024

X_train_encoded = tokenizer.batch_encode_plus(X_train_data.tolist(),
                                              padding=True,
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

X_val_encoded = tokenizer.batch_encode_plus(X_val.tolist(),
                                              padding=True,
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

X_test_encoded = tokenizer.batch_encode_plus(X_test.tolist(),
                                              padding=True,
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')


In [11]:
# SINIFLANDIRMA MODELINI YUKLEME

model = TFBertForSequenceClassification.from_pretrained('dbmdz/bert-base-turkish-cased', num_labels = 3)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# MODELI COMPILE ETME

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [13]:
# MODELI EGITME

history = model.fit(
    [X_train_encoded['input_ids'], X_train_encoded['token_type_ids'], X_train_encoded['attention_mask']],
    y_train_data,
    validation_data=(
      [X_val_encoded['input_ids'], X_val_encoded['token_type_ids'], X_val_encoded['attention_mask']],y_val),
    batch_size=16,
    epochs=5
)

Epoch 1/5
121/121 [==============================] - 316s 2s/step - loss: 0.7198 - accuracy: 0.6925 - val_loss: 0.4724 - val_accuracy: 0.8210
Epoch 2/5
121/121 [==============================] - 273s 2s/step - loss: 0.3655 - accuracy: 0.8670 - val_loss: 0.3939 - val_accuracy: 0.8541
Epoch 3/5
121/121 [==============================] - 274s 2s/step - loss: 0.2035 - accuracy: 0.9322 - val_loss: 0.4263 - val_accuracy: 0.8412
Epoch 4/5
121/121 [==============================] - 274s 2s/step - loss: 0.1279 - accuracy: 0.9653 - val_loss: 0.5432 - val_accuracy: 0.8541
Epoch 5/5
121/121 [==============================] - 274s 2s/step - loss: 0.0755 - accuracy: 0.9767 - val_loss: 0.4895 - val_accuracy: 0.8489


In [14]:
# EVALUATE MODEL

test_loss, test_accuracy = model.evaluate(
    [X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']],
    y_test
)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

31/31 [==============================] - 23s 650ms/step - loss: 0.4807 - accuracy: 0.8449
Test loss: 0.48065435886383057, Test accuracy: 0.8448810577392578


In [15]:
# MODELI KAYDETME

path = 'path-to-save'
# Save tokenizer
tokenizer.save_pretrained(path +'/Tokenizer')

# Save model
model.save_pretrained(path +'/Model')

In [16]:
# MODELI YUKLEME

# Load tokenizer
bert_tokenizer = BertTokenizer.from_pretrained(path +'/Tokenizer')

# Load model
bert_model = TFBertForSequenceClassification.from_pretrained(path +'/Model')

Some layers from the model checkpoint at path-to-save/Model were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at path-to-save/Model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [17]:
### DENEME

pred = bert_model.predict(
	[X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']])

# pred is of type TFSequenceClassifierOutput
logits = pred.logits

# Use argmax along the appropriate axis to get the predicted labels
pred_labels = tf.argmax(logits, axis=1)

# Convert the predicted labels to a NumPy array
pred_labels = pred_labels.numpy()

label = {
	0: 'Olumsuz',
	1: 'Olumlu',
  2: 'Nötr'
}

# Map the predicted labels to their corresponding strings using the label dictionary
pred_labels = [label[i] for i in pred_labels]
Actual = [label[i] for i in y_test]

print('Predicted Label :', pred_labels[:10])
print('Actual Label :', Actual[:10])


31/31 [==============================] - 24s 669ms/step
Predicted Label : ['Olumlu', 'Olumsuz', 'Olumlu', 'Nötr', 'Olumlu', 'Olumsuz', 'Nötr', 'Olumsuz', 'Nötr', 'Olumlu']
Actual Label : ['Olumlu', 'Olumsuz', 'Olumlu', 'Nötr', 'Olumsuz', 'Olumsuz', 'Nötr', 'Olumsuz', 'Nötr', 'Olumlu']


In [18]:
print("Classification Report: \n", classification_report(Actual, pred_labels))

Classification Report: 
               precision    recall  f1-score   support

        Nötr       0.71      0.57      0.64       129
      Olumlu       0.78      0.87      0.82       243
     Olumsuz       0.90      0.89      0.90       595

    accuracy                           0.84       967
   macro avg       0.80      0.78      0.78       967
weighted avg       0.84      0.84      0.84       967



In [19]:
# KULLANICI GIRISI ILE TAHMIN FONKSIYONU

def Get_sentiment(Review, Tokenizer=bert_tokenizer, Model=bert_model):
	# Convert Review to a list if it's not already a list
	if not isinstance(Review, list):
		Review = [Review]

	Input_ids, Token_type_ids, Attention_mask = Tokenizer.batch_encode_plus(Review,
																			padding=True,
																			truncation=True,
																			max_length=1024,
																			return_tensors='tf').values()
	prediction = Model.predict([Input_ids, Token_type_ids, Attention_mask])

	# Use argmax along the appropriate axis to get the predicted labels
	pred_labels = tf.argmax(prediction.logits, axis = 1)

	# Convert the TensorFlow tensor to a NumPy array and then to a list to get the predicted sentiment labels
	pred_labels = [label[i] for i in pred_labels.numpy().tolist()]
	return pred_labels


In [20]:
# KULLANICI GIRISI ILE TAHMIN FONKSIYONU

Review = "Merhaba. Vodafone nin sundugu bu hizmetlerden cok memnum kaldim. Daha once diger operatorleri denemistim ve sorunlar yasamistim. Vodafone ye gectikten sonra her sey duzeldi. Akrabalarıma da vodafone ye gecmeli konusunda ısrarcı olacagim. Fiyatlari cazip ve guzel. Tesekkurler."
Get_sentiment(Review)

1/1 [==============================] - 3s 3s/step


['Olumlu']

In [22]:
Review = "Fiyatlar cok ucuz. Cok guzel cekiyor. Tavsiye ediyorum."
Get_sentiment(Review)

1/1 [==============================] - 0s 51ms/step


['Olumlu']

In [23]:
Review = "1 haftadır arama yapamıyorum. Faturalarımı ödememe ragmen sebekeye baglanılamıyor hatasi aliyorum. Bu sorunu lutfen en erken vakitte cozun. Ticari is yapiyorum. Cok zarardayim."
Get_sentiment(Review)

1/1 [==============================] - 0s 45ms/step


['Olumsuz']

In [24]:
Review = "Vodafone yi cok seviyorum. Cok memnun kaldim. Cok tesekkur ederim. İyi ki varsiniz."
Get_sentiment(Review)

1/1 [==============================] - 0s 41ms/step


['Olumlu']

In [25]:
Review = "Cok iyi sevdim baya bunu. tesekkurler vodafone"
Get_sentiment(Review)

1/1 [==============================] - 0s 42ms/step


['Olumlu']

In [26]:
Review = "kotu berbat."
Get_sentiment(Review)

1/1 [==============================] - 0s 39ms/step


['Olumsuz']

In [27]:
Review = "Vodafone nin sundugu hizmetlerden cok memnum kaldim. Diger operatorlerden daha iyi daha guzel. Vodafone ye gecmek hayatimda aldigim en guzel karardi. Akrabalarima da vodafone ye gecmeleri konusunda ısrarcı olacagim. Fiyatlari cazip ve guzel. Tesekkur ederim."
Get_sentiment(Review)

1/1 [==============================] - 0s 69ms/step


['Olumlu']

In [28]:
Review = "Bir daha asla vodafone kullanmam."
Get_sentiment(Review)

1/1 [==============================] - 0s 59ms/step


['Olumsuz']

In [29]:
Review = "ne iyi ne de kotu. Kararsizim"
Get_sentiment(Review)

1/1 [==============================] - 0s 41ms/step


['Nötr']